In [113]:
from crewai import Agent, Crew, Process, Task, LLM
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
from langchain_openai import OpenAI

import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [114]:
XAI_API_KEY = os.getenv("XAI_API_KEY")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")
def get_llm() -> LLM:
    # llm = None
    # try:
    #     llm = LLM(
    #         model="grok-beta",
    #         base_url="https://api.x.ai/v1",
    #         api_key=XAI_API_KEY,
    #     )
    # except:
    llm=LLM(model="ollama/crewai-llama3:latest", temperature=0.7)
    # llm = OpenAI(api_key=OPENAI_API_KEY, model="gpt-4", temperature=0.3)
    return llm

In [115]:
class AgentSea:
    def __init__(self, llm, company_name: str):
        self.llm = llm
        self.company_name = company_name

    def ticker_finder_agent(self):
        return Agent(
            role="Senior Online Researcher",
            goal=
                """
                Conduct an amazing online search to find the stock ticker for {self.company_name}.
                IMPORTANT:
                - If you cannot find the result, return "Ticker not found".
                - Do not include any unnecessary words or phrases in your response and only provide the ticker symbol if found.
                """,
            backstory=
                """
                You are a Senior Online Researcher that specializes in finding a company's stock martket ticker symbol with only the company name.
                """,
            agent_prompt = """
                You are an AI agent tasked with analyzing stock data. Follow these steps:

                1. Think about what you need to do to complete the task.
                2. Choose an action from the available tools.
                3. Provide the action input as a dictionary.
                4. Observe the result of the action.
                5. Repeat steps 1-4 until you have enough information to provide a final answer.
                6. When ready to give a final answer, use the format:

                Thought: I now can give a great answer
                Final Answer: [Your comprehensive answer here]

                Remember:
                - Always start with a Thought.
                - Use each tool only once.
                - Provide your final answer in the specified format.
                """,
            max_iter=5,
            max_retry_limit=3,
            tools=[SerperDevTool(), ScrapeWebsiteTool()],
            llm=self.llm,
            verbose=True,
            memory=False
        )

In [116]:
from pydantic.v1 import BaseModel
from typing import List

class CompanyInfo(BaseModel):
    company_name: str
    ticker: str

In [117]:
class AgentTasks:
    def __init__(self, company_name: str):
        self.company_name = company_name
    
    def find_correct_company_ticker(self, agent: Agent):
        return Task(
            description=(
                f"Conduct a thorough research about the company and find the correct stock market ticker symbol for {self.company_name}."
            ),
            agent=agent,
            expeceted_output_type="A JSON object containing only the company name and ticker symbol.",
            output_json=CompanyInfo
            # context=tasks,
        )




In [119]:
inputs = {
'company_name': 'tesla'
}
llm = get_llm()
if llm is None:
    raise ValueError("LLM failed to initialize. Please check your LLM provider settings.")
else:
    print(f"LLM initialized: {llm}")
agent_sea = AgentSea(llm, inputs['company_name'])
tasks = AgentTasks(inputs['company_name'])

ticker_agent = agent_sea.ticker_finder_agent()
task = tasks.find_correct_company_ticker(ticker_agent)

LLM initialized: OpenAI
Params: {'model_name': 'gpt-4', 'temperature': 0.3, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'logit_bias': {}, 'seed': None, 'logprobs': None, 'max_tokens': 256}
AgentSea initialized: <__main__.AgentSea object at 0x000001A6FF72B0D0>


NameError: name 'company_name' is not defined